In [1]:

import pandas as pd 
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt 

In [2]:
df = pd.read_csv('data/gemstone.csv')

In [6]:
x=df.drop('price',axis=1)
y=df['price']

In [7]:
x

,id,carat,cut,color,clarity,depth,table,x,y,z
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77
...,...,...,...,...,...,...,...,...,...,...
193568,193568,0.31,Ideal,D,VVS2,61.1,56.0,4.35,4.39,2.67
193569,193569,0.70,Premium,G,VVS2,60.3,58.0,5.75,5.77,3.47
193570,193570,0.73,Very Good,F,SI1,63.1,57.0,5.72,5.75,3.62
193571,193571,0.34,Very Good,D,SI1,62.9,55.0,4.45,4.49,2.81


In [8]:
y

0         13619
1         13387
2          2772
3           666
4         14453
          ...  
193568     1130
193569     2874
193570     3036
193571      681
193572     2258
Name: price, Length: 193573, dtype: int64

In [13]:
numerical_cols=x.select_dtypes(exclude='object').columns

In [14]:
categorical_cols=x.select_dtypes(include='object').columns

In [15]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [22]:
num_pipeline = Pipeline(
    steps=[('imputer',SimpleImputer(strategy='median')),
           ('scaler',StandardScaler())])

categorical_pipeline=Pipeline(
    steps=[('encoder',OrdinalEncoder()),
           ('imputer',SimpleImputer(strategy='most_frequent'))])
           

In [26]:
preprocessor=ColumnTransformer([
('num_pipeline',num_pipeline,numerical_cols),
('categorical_pipeline',categorical_pipeline,categorical_cols)])

In [27]:
preprocessor

ColumnTransformer(transformers=[('num_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 Index(['id', 'carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')),
                                ('categorical_pipeline',
                                 Pipeline(steps=[('encoder', OrdinalEncoder()),
                                                 ('imputer',
                                                  SimpleImputer(strategy='most_frequent'))]),
                                 Index(['cut', 'color', 'clarity'], dtype='object'))])

In [29]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=30)

In [33]:
x_train=pd.DataFrame(preprocessor.fit_transform(x_train),columns=preprocessor.get_feature_names_out())

In [34]:
x_test=pd.DataFrame(preprocessor.fit_transform(x_test),columns=preprocessor.get_feature_names_out())

In [35]:
x_train.head()

,num_pipeline__id,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,categorical_pipeline__cut,categorical_pipeline__color,categorical_pipeline__clarity
0,1.274919,-0.975439,-0.849607,-0.121531,-1.042757,-1.080970,-1.123150,2.0,5.0,7.0
1,-1.102907,0.235195,1.833637,-0.121531,0.318447,0.279859,0.485354,1.0,1.0,2.0
2,-0.997613,0.494617,0.815855,0.399800,0.570855,0.606458,0.673737,3.0,3.0,4.0
3,-1.173764,-1.018676,0.260701,0.921131,-1.214034,-1.244270,-1.195605,3.0,3.0,5.0
4,-0.914812,-0.953821,-0.664555,-0.642862,-1.069801,-1.044681,-1.094168,2.0,6.0,7.0


In [36]:
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [38]:
regression=LinearRegression()
regression.fit(x_train,y_train)

LinearRegression()

In [39]:
regression.coef_

array([-8.63534937e-01,  6.57650503e+03, -2.04010420e+02, -1.64122182e+02,
       -2.07773916e+03, -4.65470661e+02, -4.83246136e+01,  1.01423423e+01,
       -2.24968186e+02,  2.41540577e+02])

In [40]:
import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [41]:
models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}
trained_model_list=[]
model_list=[]
r2_list=[]

In [43]:
for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(x_train,y_train)

    #Make Predictions
    y_pred=model.predict(x_test)
    print((y_pred.mean()))

    mae, rmse, r2_square=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')

3972.2747557702296
LinearRegression
Model Training Performance
RMSE: 1110.8809419234087
MAE: 710.3795519824416
R2 score 92.42411705832114


3972.271937056906
Lasso
Model Training Performance
RMSE: 1110.814464949226
MAE: 711.3358733165044
R2 score 92.42502373821264


3972.274536560796
Ridge
Model Training Performance
RMSE: 1110.8834568286854
MAE: 710.4009841222827
R2 score 92.42408275644624


3971.0832988010293
Elasticnet
Model Training Performance
RMSE: 1576.523567582304
MAE: 1097.0733490521804
R2 score 84.74194562910552




In [45]:
model_list

['LinearRegression', 'Lasso', 'Ridge', 'Elasticnet']